In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Parkinson

In [ ]:
import os, glob
import pandas as pd
import numpy as np
from collections import defaultdict

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [ ]:
# Rawdata 저장 경로
RAWDATAPATH = os.path.join(os.getcwd(), 'Rawdata')
RAW_CONTROL = os.path.join(RAWDATAPATH , 'Controls')  # '/content/drive/MyDrive/Parkinson/Rawdata/Controls'
RAW_PD = os.path.join(RAWDATAPATH, 'PD')              # '/content/drive/MyDrive/Parkinson/Rawdata/PD'


# 추출 후 저장할 경로
DATASETPATH = './dataset'
DATA_CONTROL = os.path.join(DATASETPATH , 'Controls')  # '/content/drive/MyDrive/Parkinson/dataset/Controls'
DATA_PD = os.path.join(DATASETPATH , 'PD')             # '/content/drive/MyDrive/Parkinson/dataset/Controls'


# CSV INFO(FW,BW 개수 및 경로) 저장 경로
SAVEPATH = './csvinfo.csv'                             # /content/drive/MyDrive/Parkinson/csvinfo.csv

## 39개 관측 센서 위치
- 머리 -> 몸통 -> 다리 -> 발
- 앞면 왼,오 -> 뒷면 왼,오 순서로!

In [ ]:
MARKERS = ['LFHD', 'RFHD', 'LBHD', 'RBHD', 'C7', 'LSHO', 'RSHO', 'CLAV', 'RBAK',
           'LUPA', 'RUPA', 'STRN', 'T10', 'LELB', 'RELB', 'LFRM', 'RFRM',
           'LWRA', 'RWRA', 'LWRB', 'RWRB', 'LFIN', 'RFIN', 'LASI', 'RASI',
           'LPSI', 'RPSI', 'LTHI', 'RTHI', 'LKNE', 'RKNE', 'LTIB', 'RTIB',
           'LANK', 'RANK', 'LTOE', 'RTOE', 'LHEE', 'RHEE']
len(MARKERS)

## 데이터 추출
- Subframe 열 제거
- 두번째 행의 측정 단위를 보면 369개의 측정값 중 1/3은 거리(mm), 1/3은 속도(mm/s), 나머지 1/3은 가속도(mm/s^2)임을 알 수 있다. \
따라서, 속도와 가속도 데이터를 제외하고 추출한다.
- CentreOfMass, CentreOfMasFloor, TURN_MARKER 제거
- 컬럼 순서 재배열 (머리 - 상체 - 하체 - 발)

- 총 컬럼 수: Frame + 마커 39개 * 3 (X,Y,Z) = 118개
- 저장 파일명: PREP_이니셜_FW/BW1-3.csv \
  (Trial 번호 1,2,3으로 다시 넘버링 함)

In [ ]:
def prep_data_from_raw(target_cate:str, target_file:str):
    """
    target_cate: 'Controls' or 'PD'
    target_file: '이니셜_FW1.csv', '이니셜_FW_01.csv', 'KangYK_FW01.csv'
    
    >> Save as './dataset/{소속}/PREP_{이니셜}_{FW/BW}{1~3}.csv' after preprocessing
    """
    print(f"\n======= Current Preprocessing: {target_file} =======")

    # ============  데이터 추출  ===============
    # 첫 두 행(Trajectories 정보) 제외하고 읽기
    # 일단 Frame을 인덱스로 설정
    target_path = RAW_PD if target_cate == 'PD' else RAW_CONTROL
    df = pd.read_csv(os.path.join(target_path, target_file), skiprows=2, index_col=0)
    
    # Sub Frame 제거
    df.drop(columns=df.columns[df.iloc[0]=='Sub Frame'], inplace=True)  

    # 단위가 mm인 데이터 추출
    df = df[df.columns[df.iloc[1]=='mm']]  # 39*3=117개 넘으면 Centre 포함
    # print(f"[Before] Number of Markers: {df.shape[1] // 3}")  # 마커 개수 확인

    # 컬럼명 변경: '이니셜:마커' or 'Unnamed: n' -> '마커명_X', '마커명_Y', '마커명_Z'
    col_names = []
    observed_markers = []
    for i, col in enumerate(df.columns):
        if i % 3 == 0:
            placement = col.split(':')[1]
            observed_markers.append(placement)
        col_names.append(placement + '_' + df.iloc[0,i])  # LFHD_X, LFHD_Y, LFHD_Z, ...
    df.columns = col_names
    
    # XYZ, 단위(mm) 행 제거
    df = df.iloc[2:]

    # CentreOfMass, CentreOfMasFloor, TURN_MARKER 제거
    df = df.loc[:, [col for col in df.columns if col.split('_')[0] in MARKERS]]
    
    # 39개 마커 중 빠진 게 있으면 Null 컬럼으로 추가
    if df.shape[1] != 117:
        missing = set(MARKERS) - set(observed_markers)
        print(f"Only {df.shape[1]} columns. Missing markers = {missing}")
        for miss in missing:
            df[miss + '_X'] = np.nan
            df[miss + '_Y'] = np.nan
            df[miss + '_Z'] = np.nan

    assert df.shape[1] == 117

    # 마커 순서 재배열
    df = df.loc[:, [m + '_' + axis for m in MARKERS for axis in ['X','Y','Z']]]

    # Frame을 인덱스에서 컬럼으로 변경
    df['Frame'] = df.index
    df = df[[df.columns[-1]] + df.columns[:-1].to_list()] # Frame 컬럼을 맨 앞으로
    df.reset_index(drop=True, inplace=True)               # 인덱스 리셋 (0부터 시작)
    assert df.shape[1] == 118


    # =============  Prep 후 csv로 저장  =============
    # 저장할 때 파일명 포맷 통일: 'PREP_이니셜_FW/BW1~3.csv'
    splitted = target_file[:-4].split('_')
    splitted.insert(0, 'PREP')
    
    if len(splitted) == 4:    # PREP_이니셜_FW_1 인 경우 (언더바 하나 더 있음)
        splitted[2] = splitted[2] + str(int(splitted[-1]))
        splitted = splitted[:3]
    assert len(splitted) == 3

    if len(splitted[-1]) > 3: # PREP_이니셜_FW01 인 경우 (숫자에 0 포함)
        splitted[-1] = splitted[-1][:2] + str(int(splitted[-1][2:]))

    # Trial Re-numbering: 124, 234, 134 등 제각각이므로 1,2,3으로 통일
    save_path = DATA_PD if target_cate == 'PD' else DATA_CONTROL
    raw_files = glob.glob(os.path.join(target_path, f"{splitted[1]}_{splitted[-1][:2]}*.csv"))
    prep_files = glob.glob(os.path.join(save_path, f"PREP_{splitted[1]}_{splitted[-1][:2]}*.csv"))
    if len(prep_files) < len(raw_files):
        splitted[-1] = splitted[-1][:2] + str(len(prep_files) + 1)
        df.to_csv(os.path.join(save_path, '_'.join(splitted) + '.csv'), encoding='utf-8', index=False)
    else:
        print(f"\n** 3 prep files of {splitted[1]}_{splitted[2][:2]} already exist!! **")

    return

In [ ]:
def generate_prep_data(rawdata_path):
    category = rawdata_path.split('/')[-1]
    for filename in sorted(os.listdir(rawdata_path)):
        prep_data_from_raw(category, filename)

### Preprocessing 수행 (Controls)

In [ ]:
# Prep 수행 전, 저장할 폴더의 존재여부 및 비어있는지 확인!!
# trial numbering 할 때 중복을 체크하지 않으므로 무조건 비어있는 상태에서 처음부터 해야함..
if not os.path.exists(DATASETPATH):
    os.mkdir(DATASETPATH)
if not os.path.exists(DATA_CONTROL):
    os.mkdir(DATA_CONTROL)

assert len(os.listdir(DATA_CONTROL)) == 0
generate_prep_data(RAW_CONTROL)

In [ ]:
print(f"[Controls] Raw: {len(os.listdir(RAW_CONTROL))} -> Prep: {len(os.listdir(DATA_CONTROL))}")
assert len(os.listdir(RAW_CONTROL)) == len(os.listdir(DATA_CONTROL))

### Preprocessing 수행 (PD)

In [ ]:
# Prep 수행 전, 저장할 폴더의 존재여부 및 비어있는지 확인!!
if not os.path.exists(DATASETPATH):
    os.mkdir(DATASETPATH)
if not os.path.exists(DATA_PD):
    os.mkdir(DATA_PD)

assert len(os.listdir(DATA_PD)) == 0
generate_prep_data(RAW_PD)

In [ ]:
print(f"[PD] Raw: {len(os.listdir(RAW_PD))} -> Prep: {len(os.listdir(DATA_PD))}")
assert len(os.listdir(RAW_PD)) == len(os.listdir(DATA_PD))

## CSV 파일 정보 취합
- `path` : 파일경로  (`./dataset/소속/PREP_이름_FW1.csv`)
- `category` : 소속  (`Controls` or `PD`)
- `name` : 환자 이니셜/이름 (`BGH` or `KimYC`)
- `trial` : 보행 실험 종류 및 횟수 (`FW1~3` or `BW1~3`)
- `frame length` : 전체 프레임 수
- `remove` : 결측률이 0.5 이상이면 `True`, 0.5 미만이면 `False`

- `LFHD` ~ `RHEE` : 각 마커별 NULL인 프레임 번호를 `시작:끝`의 범위 또는 `프레임값`으로 표현\
여러 개의 결측 구간을 공백으로 구분 --> "701:705 708"

In [ ]:
def find_null_from_prep(target_cate, direction):
    target_path = DATA_CONTROL if target_cate == 'Controls' else DATA_PD
    target_files = [f for f in os.listdir(target_path) if direction in f.split('_')[-1]]
    
    # 해당 csv의 NULL 정보를 저장할 데이터프레임
    df_info = pd.DataFrame(index=range(len(target_files)),
                           columns=[['path','category','name','trial','frame length','remove'] + MARKERS])
    df_info['category'] = target_path.split('/')[-1]

    # 해당 소속의 모든 csv 파일에 대해 NULL 탐색
    for id, filename in enumerate(sorted(target_files)):
        path = os.path.join(target_path, filename)     # ./dataset/소속/PREP_이니셜_FW/BW1~3.csv
        _, name, trial = filename[:-4].split('_')      # [PREP, 이니셜, FW/BW1~3]
        
        # 데이터 불러오기
        df = pd.read_csv(path, index_col=0)            # Frame을 인덱스로 설정
        df_info.loc[id, ['path','name','trial','frame length']] = [path, name, trial, df.shape[0]]

        # NULL 찾기: X,Y,Z 컬럼이 똑같이 결측이므로 '마커멍_X'만 본다.
        remove = False
        for col in df.columns[::3]:
            if df[col].isna().sum() == 0: continue     # 결측값이 없으면 스킵
            
            # ===== 마커에 결측값이 존재하는 경우 =====
            null = df.loc[df[col].isna()].index.values # NULL 프레임 번호 리스트

            # 전체 프레임 중 NULL 프레임 비율이 0.5 이상이면 해당 csv 파일은 제거 대상이다.
            if len(null) / len(df) >= 0.5:
                remove = True
            
            # 띄엄띄엄 결측이므로 NULL 프레임 구간들의 범위 저장 ['시작:끝', ..., '시작:끝']
            null = np.append(null, np.array([-1]))     # 인덱스 에러 방지용 더미값 -1 추가     
            null_range = []                            # NULL 프레임 범위 저장할 리스트
            i,j = 0,1                                  # i: 시작, j: 끝
            while j < len(null):
                while null[j] - null[j-1] == 1 and j < len(null)-1:
                    j += 1   # 프레임 값이 불연속일 때까지 끝 인덱스(j)를 1씩 증가
                
                if i == j-1: # 프레임 하나만 NULL인 경우 한 점만 저장
                    null_range.append(str(null[i])) 
                else:        # NULL 프레임의 구간 '시작:끝' 저장
                    null_range.append(str(null[i]) + ':' + str(null[j-1]))

                i = j        # 다음 구간 시작점 = 이전 구간 끝점 직후
                j = i+1      # 다시 j를 끝점으로 하여 다음 다음 구간 탐색
            
            # 해당 마커명에 결측 구간에 대한 문자열 저장 -> '시작:끝 포인트 시작:끝 ...'
            df_info.loc[id, col[:-2]] = ' '.join(null_range) 

        # 모든 마커에 대한 결측 탐색이 끝나면 해당 csv 파일의 제거 여부를 저장
        df_info.loc[id, 'remove'] = remove
    
    return df_info

In [ ]:
def generate_csv_info_table():
    df_Controls_BW = find_null_from_prep('Controls', 'BW')
    df_Controls_FW = find_null_from_prep('Controls', 'FW')

    df_PD_BW = find_null_from_prep('PD', 'BW')
    df_PD_FW = find_null_from_prep('PD', 'FW')

    return pd.concat([df_Controls_BW, df_Controls_FW, df_PD_BW, df_PD_FW], ignore_index=True)

In [ ]:
df_info = generate_csv_info_table()
df_info

In [ ]:
df_info.to_csv(SAVEPATH, encoding='utf-8', index=False)